In [1]:
import pandas as pd

In [2]:
lang = 'it'
#true_mentions_file = f'output/multicardioner_track2_cardioccc_dev_{lang}.tsv'
mentions_file = f'output/multicardioner_track2_cardioccc_dev_it_predictions.tsv'
result_mentions_file = f'output/multicardioner_track2_cardioccc_dev_it_predictions_joined.tsv'

In [3]:
df_mentions = pd.read_csv(mentions_file, sep='\t', header=0, index_col=False, keep_default_na=False)
df_mentions.head()

,filename,label,start_span,end_span,text
0,casos_clinicos_cardiologia10,FARMACO,2557,2569,Cefazolina2g
1,casos_clinicos_cardiologia10,FARMACO,2581,2592,gentamicina
2,casos_clinicos_cardiologia10,FARMACO,2608,2619,rifampicina
3,casos_clinicos_cardiologia10,FARMACO,3265,3274,amikacina
4,casos_clinicos_cardiologia10,FARMACO,3280,3291,vancomicina


In [4]:
# combine predictions if separated by + or /

In [5]:
df_sentences = pd.read_csv(f'data/custom_dataset_track2_{lang}/dev_sentences.tsv', sep='\t', keep_default_na=False)
df_sentences.head()

,filename,batch_number,batch_start,text
0,casos_clinicos_cardiologia10,1,0,"Anamnesi\nUomo, 79 anni."
1,casos_clinicos_cardiologia10,2,24,Autosufficiente.
2,casos_clinicos_cardiologia10,3,41,Di Salto.
3,casos_clinicos_cardiologia10,4,51,Anamnesi: -Ipertensione arteriosa cronica.
4,casos_clinicos_cardiologia10,5,94,Ex fumatore.


In [6]:
filenames = list(df_mentions['filename'].unique())
# sort by file, start, end
df_mentions = df_mentions.sort_values(['filename', 'start_span', 'end_span'])

In [7]:
def combine_entities(current_ents, sentence, sentence_start):
    if len(current_ents) == 1:
        return current_ents[0]
    # else combine entities
    entity = current_ents[0]
    entity['end_span'] = current_ents[-1]['end_span']
    entity['text'] = sentence[entity['start_span']-sentence_start:entity['end_span']-sentence_start]
    return entity

In [8]:
def get_current_sentence(start, file_sentences):
    #row['start_span'], file_sentences
    current = file_sentences[file_sentences['batch_start'] <= start].iloc[-1]
    #print(current)
    return current['batch_start'], current['text']        

In [9]:
file_sentences = df_sentences[df_sentences['filename'] == 'casos_clinicos_cardiologia10']
sent = get_current_sentence(191, file_sentences)
sent

(107,
 '-Diabete mellito di tipo 2 con ripercussioni macro e microvascolari (cardiopatia ischemica, arteriopatia del MMII con amputazione di entrambi gli alluci, malattia renale cronica, retinopatia diabetica).')

In [10]:
joined_mentions = []

for file in filenames:
    file_mentions = df_mentions[df_mentions['filename'] == file]
    file_sentences = df_sentences[df_sentences['filename'] == file]
    current_entries = []
    current_sentence_start = 0
    current_sentence_end = -1
    current_sentence_text = ''
    for index, row in file_mentions.iterrows():       
        if row['start_span'] > current_sentence_end:
            # get next sentence
            if len(current_entries) > 0:
                joined_mentions.append(combine_entities(current_entries, current_sentence_text, current_sentence_start))
                current_entries.clear()
            current_sentence_start, current_sentence_text = get_current_sentence(row['start_span'], file_sentences)
            current_sentence_end = current_sentence_start + len(current_sentence_text.rstrip())
        
        # if row in the same sentence
        if len(current_entries) == 0:
            current_entries.append(row)
        else:            
        # foreach consecutive pair - if end of first == start of second or if the only char between is +/ or whitespace
        # 2 or more consecutive entries - add new entry, once the next non-consecutive entity is found + at the end            
            if row['start_span'] - current_entries[-1]['end_span'] == 0:
                current_entries.append(row)
            else:
                text_between = current_sentence_text[current_entries[-1]['end_span']-current_sentence_start:row['start_span']-current_sentence_start]

                if text_between.strip() == '+' or text_between == '/':
                    current_entries.append(row)
                else:
                    # disjoint entities
                    joined_mentions.append(combine_entities(current_entries, current_sentence_text, current_sentence_start))
                    current_entries.clear()
                    current_entries.append(row)
    
    if len(current_entries) > 0:
        joined_mentions.append(combine_entities(current_entries, current_sentence_text, current_sentence_start))
        current_entries.clear()                

In [11]:
df_joined_mentions = pd.DataFrame.from_records(joined_mentions)

In [12]:
df_joined_mentions.head()

,filename,label,start_span,end_span,text
0,casos_clinicos_cardiologia10,FARMACO,2557,2569,Cefazolina2g
1,casos_clinicos_cardiologia10,FARMACO,2581,2592,gentamicina
2,casos_clinicos_cardiologia10,FARMACO,2608,2619,rifampicina
3,casos_clinicos_cardiologia10,FARMACO,3265,3274,amikacina
4,casos_clinicos_cardiologia10,FARMACO,3280,3291,vancomicina


In [13]:
df_joined_mentions['remove'] = df_joined_mentions['text'].apply(lambda x: x.strip() == ',' or x.strip() == '%')

In [14]:
df_joined_mentions = df_joined_mentions[~df_joined_mentions['remove']]

In [15]:
df_joined_mentions[['filename', 'label', 'start_span', 'end_span', 'text']].to_csv(result_mentions_file, sep='\t', index=False)

In [16]:
df_mentions.shape

(2642, 5)

In [17]:
df_joined_mentions.shape

(2512, 6)

In [ ]:
#|0.903|0.9008|0.9019 - en
# |0.9228|0.9139|0.9183 - es
